# Imaginary and Real Data calculations from Impedance and Phase Data



In [ ]:
import pandas as pd
import numpy as np

# dataset load
file_path = ["/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/r3.csv"]
file_loc = ["r3_ip.csv"]

new_rows = []

for j in range(1):
    df = pd.read_csv(file_path[j])   # no headers assumed
    first_col = df.iloc[:, 0].copy()
    new_rows = []  
    
    for i in range(0, len(df), 2):  
        imp_row = df.iloc[i, 1:].astype(float).values      # impedance row (skip label col0)
        phase_row = df.iloc[i+1, 1:].astype(float).values  # phase row (skip label col0)
    
        # convert phase to radians (since given in degrees)
        phase_rad = np.deg2rad(phase_row)
    
        # compute real & imag
        real_row = imp_row * np.cos(phase_rad)
        imag_row = imp_row * np.sin(phase_rad)
    
        # using first column value from original dataset
        real_label = first_col.iloc[i]   
        imag_label = first_col.iloc[i+1] 
    
        # add to new_rows with same first column + values
        new_rows.append([real_label] + real_row.tolist())
        new_rows.append([imag_label] + imag_row.tolist())
    
    # saving
    df_new = pd.DataFrame(new_rows)

    sample_numbers = np.repeat(np.arange(1, len(df_new)//2 + 1), 2)
    df_new.insert(0, "Sample", sample_numbers)
    df_new.iloc[:, 2:] = df_new.iloc[:, 2:].astype(float).round(3)
    
    df_new.to_csv(file_loc[j], index=False, header=False)
    
    print(f"Saved as {file_loc[j]} with Real & Imag rows")


Saved as r3_ip.csv with Real & Imag rows


# Dividing based on time stamps

This script divides the main database into 4 different .csv files based on the time instance of collection, renaming it in he process as tji.csv, 
- where i is the time instance of collection(0, 2, 4 and 6 min), and 
- j is the nature of data(1 - 1st set, 2 - 2nd set, and 3 - adulterated data, 3rd set).

In [ ]:
import os
import pandas as pd

input_file = "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/r3_ip_t.csv"
df = pd.read_csv(input_file)

# to ensure first column is numeric
df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce')

# Directory where input file is located
outdir = os.path.dirname(input_file)

# Mapping of values to filenames
mapping = {
    0: "t30.csv",
    2: "t32.csv",
    4: "t34.csv",
    6: "t36.csv"
}

# looping and saving subsets
for val, filename in mapping.items():
    subset = df[df.iloc[:, 1] == val]   
    outfile = os.path.join(outdir, filename)
    subset.to_csv(outfile, index=False, header=False)
    print(f"Saved {len(subset)} rows to {outfile}")


Saved 166 rows to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/t30.csv
Saved 166 rows to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/t32.csv
Saved 166 rows to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/t34.csv
Saved 166 rows to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/t36.csv


In [ ]:
import os
import pandas as pd

input_file = "/home/raginivi/Desktop/IDP_Juice_Classification/sem7/Real_IMag/r2_ip.csv"
df = pd.read_csv(input_file, header=None)

# ensure first column is numeric
df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce')

outdir = os.path.dirname(input_file)

# Mapping of values to filenames
mapping = {
    0: "t20.csv",
    2: "t22.csv",
    4: "t24.csv",
    6: "t26.csv"
}

# looping and save subsets
for val, filename in mapping.items():
    subset = df[df.iloc[:, 1] == val]   # now comparison should work
    outfile = os.path.join(outdir, filename)
    subset.to_csv(outfile, index=False, header=False)
    print(f"Saved {len(subset)} rows to {outfile}")


Saved 74 rows to /home/raginivi/Desktop/IDP_Juice_Classification/sem7/Real_IMag/t20.csv
Saved 74 rows to /home/raginivi/Desktop/IDP_Juice_Classification/sem7/Real_IMag/t22.csv
Saved 74 rows to /home/raginivi/Desktop/IDP_Juice_Classification/sem7/Real_IMag/t24.csv
Saved 74 rows to /home/raginivi/Desktop/IDP_Juice_Classification/sem7/Real_IMag/t26.csv


# Dividing based on Classes

This script is used with different set of input csvs, it is used to further divide the csvs based on the class of the data collected. 

For example here, the data is divided for the classes:
- Paper Boat and Frooti Brand, 
- and _s or _w suffix denotes the type of adulterated data as 25% by volume sugar and 12.5% sugar + 12.5% water, respectively. 

In [ ]:
import os
import pandas as pd

locations = [
    "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/t30.csv",
    "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/t32.csv",
    "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/t34.csv",
    "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/t36.csv",
]

save_dir = "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split" 
os.makedirs(save_dir, exist_ok=True)  # create if it doesn't exist

for file_path in locations:
    df = pd.read_csv(file_path, header=None)  # no headers assumed
    basename = os.path.splitext(os.path.basename(file_path))[0]  # e.g., t12
    
    # get unique class names in 3rd column 
    classes = pd.unique(df.iloc[:, 3])
    
    for cls in classes:
        # filtering rows belonging to this class
        subset = df[df.iloc[:, 3] == cls]
        
        # creating output filename
        class_letter = cls[0].upper() +cls[-1] # first letter and last letter of class
        out_file = os.path.join(save_dir, f"{basename}{class_letter}.csv")
        
        subset.to_csv(out_file, index=False, header=False)
        print(f"Saved {len(subset)} rows of class '{cls}' to {out_file}")


Saved 16 rows of class 'paperB_s' to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Ps.csv
Saved 16 rows of class 'paperB_w' to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Pw.csv
Saved 16 rows of class 'frooti_s' to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Fs.csv
Saved 14 rows of class 'frooti_w' to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Fw.csv
Saved 60 rows of class 'Frooti' to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Fi.csv
Saved 44 rows of class 'PaperBoat' to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Pt.csv
Saved 16 rows of class 'paperB_s' to /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/sp

# Subtracting the Base readings from the t timestamp readings.

Data processing by subtracting the base value from the 2, 4 and 6 min instances of data collected. 

(This was another route that was initally being researched, but left undtouched for the time being.)

In [ ]:
import pandas as pd
import os

reference_files = {
    "Fi": "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Fi.csv",
    "Fs": "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Fs.csv",
    "Fw": "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Fw.csv",
    "Pt": "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Pt.csv",
    "Ps": "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Ps.csv",
    "Pw": "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Pw.csv"
}

target_files = {
    "Fi": ["/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t32Fi.csv",
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t34Fi.csv", 
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t36Fi.csv"],
    "Fs": ["/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t32Fs.csv",
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t34Fs.csv", 
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t36Fs.csv"],
    "Fw": ["/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t32Fw.csv",
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t34Fw.csv", 
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t36Fw.csv"],
    "Pt": ["/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t32Pt.csv", 
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t34Pt.csv",
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t36Pt.csv"],
    "Ps": ["/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t32Ps.csv", 
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t34Ps.csv", 
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t36Ps.csv"],
    "Pw": ["/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t32Pw.csv", 
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t34Pw.csv", 
          "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t36Pw.csv"]
}

output_dir = "/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/subtract"
os.makedirs(output_dir, exist_ok=True)

for cls, ref_path in reference_files.items():
    ref_df = pd.read_csv(ref_path, header=None)   # no headers, keep all rows
    ref_meta = ref_df.iloc[:, :4]                # first 4 columns untouched
    ref_data = ref_df.iloc[:, 4:].astype(float)  # numeric part
    
    for tgt_path in target_files[cls]:
        tgt_df = pd.read_csv(tgt_path, header=None)
        tgt_meta = tgt_df.iloc[:, :4]  
        tgt_data = tgt_df.iloc[:, 4:].astype(float)
        
        # Cell-wise subtraction
        result_data = tgt_data.values - ref_data.values
        
        # Rebuild full DataFrame: metadata + subtracted values
        result_df = pd.concat([tgt_meta, pd.DataFrame(result_data)], axis=1)
        
        # Save
        out_name = os.path.basename(tgt_path).replace(".csv", "_subtracted.csv")
        result_df.to_csv(os.path.join(output_dir, out_name), index=False, header=False)

print("Subtraction complete, all rows included.")


Subtraction complete, all rows included.


## Splitting into Real and Imag Datasets

Processed csvs are divided into real of imaginary only data, and saved as _r.csv or _i.csv denoting real an imaginary data csv, respectively.

Further, the scripts are stored into respective real and iimaginary folders for better arrangment and accesibility of the data.

In [ ]:
import pandas as pd
import os
import glob

def split_real_imag_folder(input_folder, output_folder):
    """
    Splits all CSV files in a folder into real/imag CSVs based on the 3rd column.
    """
    os.makedirs(output_folder, exist_ok=True)

    csv_files = glob.glob(os.path.join(input_folder, "*.csv"))
    
    if not csv_files:
        print("No CSV files found in", input_folder)
        return

    for file in csv_files:
        df = pd.read_csv(file, header=None)  # <-- no header

        # splitting based on 3rd column (index=2)
        real_df = df[df.iloc[:, 2].str.lower() == "real"]
        imag_df = df[df.iloc[:, 2].str.lower() == "imag"]

        base_name = os.path.splitext(os.path.basename(file))[0]
        real_path = os.path.join(output_folder, f"{base_name}_r.csv")
        imag_path = os.path.join(output_folder, f"{base_name}_i.csv")

        real_df.to_csv(real_path, index=False, header=False)
        imag_df.to_csv(imag_path, index=False, header=False)

        print(f"Processed {file}: {len(real_df)} real rows -> {real_path}, {len(imag_df)} imag rows -> {imag_path}")


# usage
split_real_imag_folder(
    input_folder="/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split",
    output_folder="/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/here"
)


Processed /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t36Fs.csv: 8 real rows -> /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/here/t36Fs_r.csv, 8 imag rows -> /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/here/t36Fs_i.csv
Processed /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t30Fs.csv: 8 real rows -> /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/here/t30Fs_r.csv, 8 imag rows -> /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/here/t30Fs_i.csv
Processed /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/t34Ps.csv: 8 real rows -> /home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/here/t34

In [ ]:
import os
import glob
import shutil

def sort_real_imag_files(input_folder, output_folder):
    """
    Moves CSV files ending with '_r.csv' into a 'real' folder
    and '_i.csv' into an 'imag' folder.
    """

    real_folder = os.path.join(output_folder, "real")
    imag_folder = os.path.join(output_folder, "imag")

    # make sure they exist
    os.makedirs(real_folder, exist_ok=True)
    os.makedirs(imag_folder, exist_ok=True)

    # get all CSVs
    csv_files = glob.glob(os.path.join(input_folder, "*.csv"))

    if not csv_files:
        print("No CSV files found in", input_folder)
        return

    for file in csv_files:
        base_name = os.path.basename(file)

        if base_name.endswith("_r.csv"):
            dest = os.path.join(real_folder, base_name)
            shutil.move(file, dest)
            print(f"Moved {base_name} -> real/")
        elif base_name.endswith("_i.csv"):
            dest = os.path.join(imag_folder, base_name)
            shutil.move(file, dest)
            print(f"Moved {base_name} -> imag/")
        else:
            print(f"Skipped {base_name} (doesn't match pattern)")

sort_real_imag_files(
    input_folder="/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/here",
    output_folder="/home/raginivi/Desktop/IDP_Juice_Classification/FruitBrandClassification/Real_IMag/alt_data/split/here"
)


Moved t30Fw_i.csv -> imag/
Moved t34Fi_r.csv -> real/
Moved t32Fw_r.csv -> real/
Moved t34Pt_i.csv -> imag/
Moved t32Fw_i.csv -> imag/
Moved t30Pt_i.csv -> imag/
Moved t30Pw_r.csv -> real/
Moved t36Fi_i.csv -> imag/
Moved t36Pt_r.csv -> real/
Moved t32Pt_i.csv -> imag/
Moved t30Fw_r.csv -> real/
Moved t36Fs_r.csv -> real/
Moved t34Fs_r.csv -> real/
Moved t32Fs_i.csv -> imag/
Moved t30Fs_r.csv -> real/
Moved t32Pt_r.csv -> real/
Moved t30Fi_r.csv -> real/
Moved t30Ps_r.csv -> real/
Moved t36Pw_r.csv -> real/
Moved t34Fs_i.csv -> imag/
Moved t36Fw_r.csv -> real/
Moved t32Fi_i.csv -> imag/
Moved t36Pt_i.csv -> imag/
Moved t30Fi_i.csv -> imag/
Moved t32Fs_r.csv -> real/
Moved t34Pw_i.csv -> imag/
Moved t34Ps_r.csv -> real/
Moved t34Fi_i.csv -> imag/
Moved t32Pw_i.csv -> imag/
Moved t36Ps_i.csv -> imag/
Moved t36Pw_i.csv -> imag/
Moved t32Fi_r.csv -> real/
Moved t34Pt_r.csv -> real/
Moved t30Fs_i.csv -> imag/
Moved t34Fw_r.csv -> real/
Moved t32Ps_r.csv -> real/
Moved t32Pw_r.csv -> real/
M